In [1]:
import pandas as pd
import requests, json
import time

In [4]:
pharmacy = pd.read_csv("./pharmacies.csv")
pharmacy

,id,name,address,city_name,county_name,town_name,x,y,data_type
0,3,청담장수 한약국,"서울특별시 강남구 도산대로89길 3, (청담동, 2층 일부)",서울,강남구,청담동,127.028171,37.525831,약국
1,4,대도약국,"서울특별시 종로구 우정국로 38-1, 1층 (견지동)",서울,종로구,견지동,127.028171,37.525831,약국
2,5,메디팜정연약국,"서울특별시 강남구 강남대로 256, 103호 (도곡동, 대우양재디오빌)",서울,강남구,도곡동,127.028171,37.525831,약국
3,6,진선약국,"서울특별시 강남구 선릉로 424, (대치동)",서울,강남구,대치동,127.028171,37.525831,약국
4,7,국민약국,"서울특별시 강서구 곰달래로 225, 1,2층 (화곡동)",서울,강서구,화곡동,127.028171,37.525831,약국
...,...,...,...,...,...,...,...,...,...
5550,11266,고바우약국,"서울특별시 송파구 마천로41길 21, (마천동)",서울,송파구,마천동,127.028171,37.525831,약국
5551,11267,선약국,"서울특별시 강서구 강서로 194-9, 1층 (화곡동)",서울,강서구,화곡동,127.028171,37.525831,약국
5552,11268,최고약국,"서울특별시 서초구 서초대로 351, 최상산부인과 202호 (서초동)",서울,서초구,서초동,127.028171,37.525831,약국
5553,11269,온누리큰사랑약국,"서울특별시 강서구 강서로 64, (화곡동)",서울,강서구,화곡동,127.028171,37.525831,약국


In [15]:
pharmacy.isnull().sum()

id                 0
name               0
address            0
city_name          0
county_name        0
town_name          0
x                  0
y                  0
data_type          0
cleaned_address    0
dtype: int64

## 정규표현식으로 address 정리

In [8]:
sample = "서울특별시 강남구 도산대로89길 3, (청담동, 2층 일부)"
result = re.sub(r',.*', '', sample)
result

'서울특별시 강남구 도산대로89길 3'

In [9]:
import re

# 정규표현식을 이용하여 각 주소에서 ,를 기준으로 뒷부분을 지우는 함수
def clean_address(address):
    return re.sub(r',.*', '', address)

In [13]:
from tqdm import tqdm

tqdm.pandas()
# progress_apply를 사용하여 address 열에 대해 clean_address 함수를 적용하여 수정된 주소를 새로운 열에 저장
pharmacy['cleaned_address'] = pharmacy['address'].progress_apply(lambda x: clean_address(x))

100%|███████████████████████████████████████████████████████████████████████████| 5555/5555 [00:00<00:00, 610042.64it/s]


In [14]:
pharmacy

,id,name,address,city_name,county_name,town_name,x,y,data_type,cleaned_address
0,3,청담장수 한약국,"서울특별시 강남구 도산대로89길 3,",서울,강남구,청담동,127.028171,37.525831,약국,서울특별시 강남구 도산대로89길 3
1,4,대도약국,"서울특별시 종로구 우정국로 38-1, 1층",서울,종로구,견지동,127.028171,37.525831,약국,서울특별시 종로구 우정국로 38-1
2,5,메디팜정연약국,"서울특별시 강남구 강남대로 256, 103호",서울,강남구,도곡동,127.028171,37.525831,약국,서울특별시 강남구 강남대로 256
3,6,진선약국,"서울특별시 강남구 선릉로 424,",서울,강남구,대치동,127.028171,37.525831,약국,서울특별시 강남구 선릉로 424
4,7,국민약국,"서울특별시 강서구 곰달래로 225, 1,2층",서울,강서구,화곡동,127.028171,37.525831,약국,서울특별시 강서구 곰달래로 225
...,...,...,...,...,...,...,...,...,...,...
5550,11266,고바우약국,"서울특별시 송파구 마천로41길 21,",서울,송파구,마천동,127.028171,37.525831,약국,서울특별시 송파구 마천로41길 21
5551,11267,선약국,"서울특별시 강서구 강서로 194-9, 1층",서울,강서구,화곡동,127.028171,37.525831,약국,서울특별시 강서구 강서로 194-9
5552,11268,최고약국,"서울특별시 서초구 서초대로 351, 최상산부인과 202호",서울,서초구,서초동,127.028171,37.525831,약국,서울특별시 서초구 서초대로 351
5553,11269,온누리큰사랑약국,"서울특별시 강서구 강서로 64,",서울,강서구,화곡동,127.028171,37.525831,약국,서울특별시 강서구 강서로 64


## address -> 법정동 (by. kakao api)

In [16]:
def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    headers = {"Authorization": "KakaoAK 905ba1855aa03b3416e81d27e5331850"}
    api_json = json.loads(str(requests.get(url,headers=headers).text))
    return api_json

In [31]:
sample = "서울특별시 강남구 도산대로89길 3"

result = get_location(sample)
result["documents"][0]["address"]["h_code"]

'1168056500'

In [17]:
sampled_df = pharmacy.sample(n=10, random_state=1234)
sampled_df

,id,name,address,city_name,county_name,town_name,x,y,data_type,cleaned_address
419,696,대교약국,"서울특별시 영등포구 여의대방로 383,",서울,영등포구,여의도동,127.028171,37.525831,약국,서울특별시 영등포구 여의대방로 383
1413,2201,노약국,"서울특별시 강북구 도봉로 174, 1층 101호",서울,강북구,미아동,127.028171,37.525831,약국,서울특별시 강북구 도봉로 174
368,614,3층한사랑약국,"서울특별시 서초구 반포대로 291, 3층 307호",서울,서초구,반포동,127.028171,37.525831,약국,서울특별시 서초구 반포대로 291
5242,10126,나무약국,"서울특별시 영등포구 도림로 158, 대림빌딩",서울,영등포구,대림동,127.028171,37.525831,약국,서울특별시 영등포구 도림로 158
5202,10052,우정약국,"서울특별시 중랑구 망우로 428, 우정약국 1층",서울,중랑구,망우동,127.028171,37.525831,약국,서울특별시 중랑구 망우로 428
2126,3129,하늘약국,"서울특별시 종로구 종로 19, 르메이에르종로타운1 2층 226-2호",서울,종로구,종로1가,127.028171,37.525831,약국,서울특별시 종로구 종로 19
2217,3254,보광프라자약국,"서울특별시 용산구 보광로 35,",서울,용산구,보광동,127.028171,37.525831,약국,서울특별시 용산구 보광로 35
387,650,타워약국,"서울특별시 중구 퇴계로 447, B동 318호",서울,중구,황학동,127.028171,37.525831,약국,서울특별시 중구 퇴계로 447
1648,2508,아침햇살약국,"서울특별시 구로구 시흥대로 561, 동해빌딩 1층",서울,구로구,구로동,127.028171,37.525831,약국,서울특별시 구로구 시흥대로 561
5349,10877,한중약국,"서울특별시 영등포구 도림로 144, 88월드타워 104호",서울,영등포구,대림동,127.028171,37.525831,약국,서울특별시 영등포구 도림로 144


### 법정동 뽑기

In [26]:
def get_dong_name(row, dong_error_list):
    try:
        location = get_location(row["address"])
        return location["documents"][0]["address"]["region_3depth_name"]
    except IndexError:
        dong_error_list.append(row["address"])
        return None

In [27]:
tqdm.pandas()
dong_error_list = []
pharmacy["town_name"] = pharmacy.progress_apply(lambda row: get_dong_name(row, dong_error_list), axis=1)

100%|███████████████████████████████████████████████████████████████████████████████| 5555/5555 [11:10<00:00,  8.29it/s]


In [28]:
pharmacy

,id,name,address,city_name,county_name,town_name,x,y,data_type,cleaned_address
0,3,청담장수 한약국,"서울특별시 강남구 도산대로89길 3,",서울,강남구,청담동,127.028171,37.525831,약국,서울특별시 강남구 도산대로89길 3
1,4,대도약국,"서울특별시 종로구 우정국로 38-1, 1층",서울,종로구,견지동,127.028171,37.525831,약국,서울특별시 종로구 우정국로 38-1
2,5,메디팜정연약국,"서울특별시 강남구 강남대로 256, 103호",서울,강남구,도곡동,127.028171,37.525831,약국,서울특별시 강남구 강남대로 256
3,6,진선약국,"서울특별시 강남구 선릉로 424,",서울,강남구,대치동,127.028171,37.525831,약국,서울특별시 강남구 선릉로 424
4,7,국민약국,"서울특별시 강서구 곰달래로 225, 1,2층",서울,강서구,None,127.028171,37.525831,약국,서울특별시 강서구 곰달래로 225
...,...,...,...,...,...,...,...,...,...,...
5550,11266,고바우약국,"서울특별시 송파구 마천로41길 21,",서울,송파구,마천동,127.028171,37.525831,약국,서울특별시 송파구 마천로41길 21
5551,11267,선약국,"서울특별시 강서구 강서로 194-9, 1층",서울,강서구,화곡동,127.028171,37.525831,약국,서울특별시 강서구 강서로 194-9
5552,11268,최고약국,"서울특별시 서초구 서초대로 351, 최상산부인과 202호",서울,서초구,서초동,127.028171,37.525831,약국,서울특별시 서초구 서초대로 351
5553,11269,온누리큰사랑약국,"서울특별시 강서구 강서로 64,",서울,강서구,화곡동,127.028171,37.525831,약국,서울특별시 강서구 강서로 64


In [29]:
pharmacy.isnull().sum()

id                   0
name                 0
address              0
city_name            0
county_name          0
town_name          134
x                    0
y                    0
data_type            0
cleaned_address      0
dtype: int64

### 행정동 코드 뽑기

In [32]:
def get_h_cd(row, h_cd_error):
    try:
        location = get_location(row["address"])
        return location["documents"][0]["address"]["h_code"]
    except IndexError:
        h_cd_error.append(row["address"])
        return None

In [33]:
tqdm.pandas()
h_cd_error = []
pharmacy["h_code"] = pharmacy.progress_apply(lambda row: get_h_cd(row, h_cd_error), axis=1)

100%|███████████████████████████████████████████████████████████████████████████████| 5555/5555 [11:23<00:00,  8.13it/s]


In [34]:
pharmacy.isnull().sum()

id                   0
name                 0
address              0
city_name            0
county_name          0
town_name          134
x                    0
y                    0
data_type            0
cleaned_address      0
h_code             134
dtype: int64

### x 좌표(경도) 뽑기

In [35]:
def get_x_point(row, x_point_error):
    try:
        location = get_location(row["address"])
        return location["documents"][0]["address"]["x"]
    except IndexError:
        x_point_error.append(row["address"])
        return None

In [36]:
tqdm.pandas()
x_point_error = []
pharmacy["x"] = pharmacy.progress_apply(lambda row: get_x_point(row, x_point_error), axis=1)

100%|███████████████████████████████████████████████████████████████████████████████| 5555/5555 [11:26<00:00,  8.09it/s]


In [37]:
pharmacy.isnull().sum()

id                   0
name                 0
address              0
city_name            0
county_name          0
town_name          134
x                  134
y                    0
data_type            0
cleaned_address      0
h_code             134
dtype: int64

In [75]:
x_point_error

['서울특별시 강서구 곰달래로 225, 1,2층 ',
 '서울특별시 강서구 마곡중앙4로 74, 104~105호 ',
 '서울특별시 용산구 원효로 51, 250,251,252,253호 ',
 '서울특별시 성동구 독서당로 175, 101-2호 ',
 '서울특별시 은평구 진관2로 29-21, 113,114호 ',
 '서울특별시 마포구 마포대로 68, 210-2호 ',
 '서울특별시 강남구 개포로 615, 석탑프라자 1층 112,113,114호 ',
 '서울특별시 송파구 송파대로 345, 1A동 2040호 ',
 '서울특별시 중구 충무로 3, 225,226,227호 ',
 '서울특별시 금천구 가산디지털1로 131, 117-1호 ',
 '서울특별시 용산구 효창원로 227, 근린3상가동 2층 201호 ',
 '서울특별시 동작구 동작대로 89, 107,108,S-12,S-13호 ',
 '서울특별시 서대문구 수색로 100, 3-7-1호 ',
 '서울특별시 용산구 이촌로 224, 210-1, 209, 217호 ',
 '서울특별시 강서구 강서로 52, 104,105호 ',
 '서울특별시 노원구 마들로 111, 207, 208호 ',
 '서울특별시 마포구 마포대로 195, 4단지상가동 219호 ',
 '서울특별시 서초구 사평대로 205, 4,5층 BP4-1A호 ',
 '서울특별시 성북구 동소문로20길 43-4, 1층 ',
 '서울특별시 종로구 대학로5길 57, 1,2층 ',
 '서울특별시 중구 장충단로 195, 1, 지하1층 ',
 '서울특별시 송파구 석촌호수로 74, 101-1호 ',
 '서울특별시 서초구 남부순환로356길 15, 111,112,126호 ',
 '서울특별시 강남구 삼성로 14, 상가동 3층 303 일부호 ',
 '서울특별시 동대문구 경희대로 17, 1,2층 ',
 '서울특별시 은평구 은평터널로 65, 1-3, 2-2호 ',
 '서울특별시 강남구 봉은사로 471, 204-1호 ',
 '서울특별시 성동구 왕십리로 410, 상가L동 221-1, 222호 ',
 '서울

### y좌표 뽑기

In [38]:
def get_y_point(row, y_point_error):
    try:
        location = get_location(row["address"])
        return location["documents"][0]["address"]["y"]
    except IndexError:
        y_point_error.append(row["address"])
        return None

In [41]:
tqdm.pandas()
y_point_error = []
pharmacy["y"] = pharmacy.progress_apply(lambda row: get_y_point(row, y_point_error), axis=1)

100%|███████████████████████████████████████████████████████████████████████████████| 5555/5555 [11:19<00:00,  8.18it/s]


In [42]:
pharmacy.isnull().sum()

id                   0
name                 0
address              0
city_name            0
county_name          0
town_name          134
x                  134
y                  134
data_type            0
cleaned_address      0
h_code             134
dtype: int64

### town_name 결측치 처리

In [90]:
pharmacy[pharmacy["town_name"].isnull()]

,id,name,address,city_name,county_name,town_name,x,y,data_type,cleaned_address,h_code


In [89]:
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 송파구 토성로 11", 'town_name'] = "풍납동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 양천구 목동남로4길 2", 'town_name'] = "신정동"

In [84]:
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 은평구 진관2로 57", 'town_name'] = "진관동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 중구 명동8가길 20", 'town_name'] = "충무로2가"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강남구 영동대로114길 56", 'town_name'] = "삼성동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강서구 양천로 731", 'town_name'] = "염창동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 도봉구 노해로65길 17-9", 'town_name'] = "창동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 마포구 만리재로 15", 'town_name'] = "공덕동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 성북구 화랑로 76", 'town_name'] = "하월곡동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 관악구 신림로 272", 'town_name'] = "신림동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강동구 진황도로 85", 'town_name'] = "길동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 노원구 섬밭로 229", 'town_name'] = "하계동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 성북구 길음로 33", 'town_name'] = "길음동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 중구 명동길 45", 'town_name'] = "명동1가"

In [82]:
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 서대문구 연세로 40", 'town_name'] = "창천동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 용산구 한강대로 95", 'town_name'] = "한강로2가"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 도봉구 마들로 650", 'town_name'] = "방학동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강남구 도곡로 242", 'town_name'] = "도곡동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 종로구 경교장길 35", 'town_name'] = "평동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강동구 아리수로93길 27", 'town_name'] = "강일동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 중구 퇴계로 213", 'town_name'] = "충무로4가"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강서구 공항대로 269-15", 'town_name'] = "마곡동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강서구 공항대로 168", 'town_name'] = "마곡동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강서구 공항대로 248", 'town_name'] = "마곡동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 성동구 행당로 84", 'town_name'] = "행당동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강서구 마곡중앙5로 81", 'town_name'] = "마곡동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 서초구 사평대로56길 7", 'town_name'] = "서초동"

In [80]:
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 종로구 종로 204", 'town_name'] = "종로4가"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 노원구 덕릉로 780", 'town_name'] = "상계동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 송파구 송파대로 111", 'town_name'] = "문정동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 중랑구 사가정로 404", 'town_name'] = "면목동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 구로구 새말로 89", 'town_name'] = "구로동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강동구 동남로49길 21-15", 'town_name'] = "둔촌동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강서구 양천로 401", 'town_name'] = "가양동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강남구 논현로 503", 'town_name'] = "역삼동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 동작구 만양로 84", 'town_name'] = "노량진동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 성북구 종암로 167", 'town_name'] = "하월곡동"

In [78]:
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 노원구 동일로 1361", 'town_name'] = "상계동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강남구 일원로 95", 'town_name'] = "일원동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 중랑구 봉우재로 66", 'town_name'] = "면목동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강남구 학동로 342", 'town_name'] = "논현동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 중랑구 공릉로2길 8-4", 'town_name'] = "묵동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 마포구 월드컵로 111", 'town_name'] = "망원동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 동대문구 왕산로 185", 'town_name'] = "청량리동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 중구 마른내로 12", 'town_name'] = "저동2가"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 성북구 동소문로7길 2", 'town_name'] = "동소문동1가"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 중구 명동7길 14", 'town_name'] = "명동1가"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 영등포구 경인로102길 13", 'town_name'] = "영등포동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강남구 밤고개로1길 10", 'town_name'] = "수서동"

In [64]:
pharmacy[pharmacy["town_name"]=="하왕십리동"]

,id,name,address,city_name,county_name,town_name,x,y,data_type,cleaned_address,h_code
10,14,왕십리약국,"서울특별시 성동구 무학봉길 92,",서울,성동구,하왕십리동,37.5622045860266,37.5622045860266,약국,서울특별시 성동구 무학봉길 92,1120052000
938,1492,유연약국,"서울특별시 성동구 왕십리로 410, 상가 L동 3층 318호",서울,성동구,하왕십리동,37.5666029325047,37.5666029325047,약국,서울특별시 성동구 왕십리로 410,1120053500
942,1496,정성온누리약국,"서울특별시 성동구 왕십리로 410, 상가L동 3층 319호",서울,성동구,하왕십리동,37.5666029325047,37.5666029325047,약국,서울특별시 성동구 왕십리로 410,1120053500
943,1497,새봄약국,"서울특별시 성동구 왕십리로 410, 상가L동 2층 210-1호",서울,성동구,하왕십리동,37.5666029325047,37.5666029325047,약국,서울특별시 성동구 왕십리로 410,1120053500
964,1521,바른약국,"서울특별시 성동구 왕십리로 410, G동 204호",서울,성동구,하왕십리동,37.5666029325047,37.5666029325047,약국,서울특별시 성동구 왕십리로 410,1120053500
965,1522,늘푸른약국,"서울특별시 성동구 왕십리로 410, D동 비103호",서울,성동구,하왕십리동,37.5666029325047,37.5666029325047,약국,서울특별시 성동구 왕십리로 410,1120053500
967,1524,송송약국,"서울특별시 성동구 왕십리로 410, 106호",서울,성동구,하왕십리동,37.5666029325047,37.5666029325047,약국,서울특별시 성동구 왕십리로 410,1120053500
974,1531,1층이화약국,"서울특별시 성동구 왕십리로 410, L동 128-1호",서울,성동구,하왕십리동,37.5666029325047,37.5666029325047,약국,서울특별시 성동구 왕십리로 410,1120053500
1339,2052,우인약국,"서울특별시 성동구 왕십리로 339,",서울,성동구,하왕십리동,37.5629076652687,37.5629076652687,약국,서울특별시 성동구 왕십리로 339,1120052000
1486,2291,대중약국,"서울특별시 성동구 왕십리로 379,",서울,성동구,하왕십리동,37.5645494848044,37.5645494848044,약국,서울특별시 성동구 왕십리로 379,1120052000


In [76]:
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 동작구 동작대로29길 69", 'town_name'] = "사당동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 중랑구 공릉로 8", 'town_name'] = "묵동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 광진구 아차산로69길 8", 'town_name'] = "광장동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 송파구 올림픽로 119", 'town_name'] = "잠실동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 동대문구 사가정로 148", 'town_name'] = "전농동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 서초구 사임당로 151", 'town_name'] = "서초동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 동대문구 서울시립대로 31", 'town_name'] = "전농동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 금천구 금하로 763-12", 'town_name'] = "시흥동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 종로구 새문안로 21", 'town_name'] = "평동"

In [71]:
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 서대문구 이화여대8길 62", 'town_name'] = "북아현동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 노원구 동일로 1701", 'town_name'] = "상계동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 서초구 바우뫼로 23", 'town_name'] = "우면동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강남구 압구정로 151", 'town_name'] = "압구정동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 금천구 가산디지털1로 186", 'town_name'] = "가산동"

In [69]:
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 중랑구 신내로 211", 'town_name'] = "신내동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강동구 양재대로 1325", 'town_name'] = "성내동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 종로구 대학로 119", 'town_name'] = "명륜4가"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강남구 삼성로 155", 'town_name'] = "대치동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 은평구 진관2로 15-46", 'town_name'] = "진관동"

pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 중구 퇴계로 447", 'town_name'] = "황학동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강서구 화곡로 197", 'town_name'] = "화곡동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 종로구 경교장길 5", 'town_name'] = "교남동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 양천구 목동서로 67", 'town_name'] = "목동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 금천구 금하로 763", 'town_name'] = "시흥동"

In [66]:
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강남구 삼성로 14", 'town_name'] = "개포동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 동대문구 경희대로 17", 'town_name'] = "회기동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 은평구 은평터널로 65", 'town_name'] = "수색동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강남구 봉은사로 471", 'town_name'] = "삼성동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 성동구 왕십리로 410", 'town_name'] = "하왕십리동"

pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 구로구 가마산로 208", 'town_name'] = "구로동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 금천구 한내로 69-15", 'town_name'] = "독산동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 도봉구 노해로 389", 'town_name'] = "창동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 양천구 신정로 293", 'town_name'] = "신정동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 노원구 동일로 1379", 'town_name'] = "상계동"

In [61]:
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 성북구 동소문로20길 43-4", 'town_name'] = "동선동1가"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 종로구 대학로5길 57", 'town_name'] = "연건동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 중구 장충단로 195", 'town_name'] = "쌍림동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 송파구 석촌호수로 74", 'town_name'] = "잠실동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 서초구 남부순환로356길 15", 'town_name'] = "양재동"

pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 동작구 흑석로 108", 'town_name'] = "흑석동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 서초구 효령로 237", 'town_name'] = "서초동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 금천구 시흥대로 225", 'town_name'] = "시흥동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 송파구 위례광장로 200", 'town_name'] = "장지동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 송파구 강동대로 57", 'town_name'] = "풍납동"

In [58]:
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 용산구 이촌로 224", 'town_name'] = "이촌동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강서구 강서로 52", 'town_name'] = "화곡동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 노원구 마들로 111", 'town_name'] = "월계동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 마포구 마포대로 195", 'town_name'] = "아현동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 서초구 사평대로 205", 'town_name'] = "반포동"

pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 종로구 종로 19", 'town_name'] = "청진동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 은평구 통일로 1034", 'town_name'] = "진관동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 서대문구 신촌로35길 10", 'town_name'] = "북아현동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 노원구 공릉로43길 1", 'town_name'] = "공릉동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 서초구 사평대로45길 10-25", 'town_name'] = "반포동"

In [55]:
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 중구 충무로 3", 'town_name'] = "필동1가"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 금천구 가산디지털1로 131", 'town_name'] = "가산동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 용산구 효창원로 227", 'town_name'] = "효창동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 동작구 동작대로 89", 'town_name'] = "사당동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 서대문구 수색로 100", 'town_name'] = "북가좌동"

In [50]:
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 성동구 독서당로 175", 'town_name'] = "옥수동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 은평구 진관2로 29-21", 'town_name'] = "진관동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 마포구 마포대로 68", 'town_name'] = "도화동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강남구 개포로 615", 'town_name'] = "개포동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 송파구 송파대로 345", 'town_name'] = "가락동"

In [51]:
# 주어진 조건에 해당하는 행의 "town_name"을 변경
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강서구 곰달래로 225", 'town_name'] = "화곡동"

In [52]:
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 강서구 마곡중앙4로 74", 'town_name'] = "마곡동"
pharmacy.loc[pharmacy['cleaned_address'] == "서울특별시 용산구 원효로 51", 'town_name'] = "원효로4가"

In [91]:
pharmacy.to_csv("./pharmacies.csv", index=False)

### h_code, x, y 좌표 결측치 처리

In [2]:
pharmacy = pd.read_csv("./pharmacies.csv")
pharmacy

,id,name,address,city_name,county_name,town_name,x,y,data_type,cleaned_address,h_code
0,3,청담장수 한약국,"서울특별시 강남구 도산대로89길 3,",서울,강남구,청담동,37.525057,37.525057,약국,서울특별시 강남구 도산대로89길 3,1.168056e+09
1,4,대도약국,"서울특별시 종로구 우정국로 38-1, 1층",서울,종로구,견지동,37.572483,37.572483,약국,서울특별시 종로구 우정국로 38-1,1.111062e+09
2,5,메디팜정연약국,"서울특별시 강남구 강남대로 256, 103호",서울,강남구,도곡동,37.486487,37.486487,약국,서울특별시 강남구 강남대로 256,1.168066e+09
3,6,진선약국,"서울특별시 강남구 선릉로 424,",서울,강남구,대치동,37.502910,37.502910,약국,서울특별시 강남구 선릉로 424,1.168063e+09
4,7,국민약국,"서울특별시 강서구 곰달래로 225, 1,2층",서울,강서구,화곡동,NaN,NaN,약국,서울특별시 강서구 곰달래로 225,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5550,11266,고바우약국,"서울특별시 송파구 마천로41길 21,",서울,송파구,마천동,37.497121,37.497121,약국,서울특별시 송파구 마천로41길 21,1.171055e+09
5551,11267,선약국,"서울특별시 강서구 강서로 194-9, 1층",서울,강서구,화곡동,37.544048,37.544048,약국,서울특별시 강서구 강서로 194-9,1.150062e+09
5552,11268,최고약국,"서울특별시 서초구 서초대로 351, 최상산부인과 202호",서울,서초구,서초동,37.495795,37.495795,약국,서울특별시 서초구 서초대로 351,1.165053e+09
5553,11269,온누리큰사랑약국,"서울특별시 강서구 강서로 64,",서울,강서구,화곡동,37.533504,37.533504,약국,서울특별시 강서구 강서로 64,1.150059e+09


In [3]:
pharmacy.isnull().sum()

id                   0
name                 0
address              0
city_name            0
county_name          0
town_name            0
x                  134
y                  134
data_type            0
cleaned_address      0
h_code             134
dtype: int64

#### h_code 결측치 처리

In [4]:
hdong_code_error = [
    1150057000,
    1150061100,
    1117057000,
    1120064500,
    1138069000,
    1144058500,
    1168067500,
    1171063100,
    1114057000,
    1154551000,
    1117058000,
    1159062000,
    1141071000,
    1117063000,
    1150059300,
    1135058000,
    1144055500,
    1165058100,
    1129057500,
    1111064000,
    1114059000,
    1171065000,
    1165065100,
    1168067000,
    1123071000,
    1138065000,
    1168059000,
    1120053500,
    1150061100,
    1126069000,
    1174066000,
    1111065000,
    1168060000,
    1138069000,
    1171052000,
    1147068000,
    1138069000,
    1114055000,
    1168058000,
    1150051000,
    1168053100,
    1132051500,
    1144056500,
    1129072500,
    1162064500,
    1174068500,
    1135061200,
    1141071000,
    1126069000,
    1129066000,
    1114055000,
    1141058500,
    1117062500,
    1132069000,
    1168065500,
    1111058000,
    1174051500,
    1114059000,
    1150060300,
    1150061100,
    1150061100,
    1120057000,
    1150063000,
    1165053100,
    1111061500,
    1135066500,
    1171063100,
    1171064200,
    1171063100,
    1126057000,
    1153056000,
    1171063100,
    1174070000,
    1150060300,
    1168064000,
    1159051000,
    1129068500,
    1135069500,
    1168072000,
    1126052000,
    1168053100,
    1126062000,
    1171063100,
    1171063100,
    1171063100,
    1144070000,
    1123070500,
    1114060500,
    1129052500,
    1114055000,
    1171063100,
    1156053500,
    1168075000,
    1159063000,
    1159063000,
    1126062000,
    1121581000,
    1114067000,
    1171067000,
    1123057000,
    1144058500,
    1144055500,
    1165052000,
    1123056000,
    1154568000,
    1111058000,
    1141055500,
    1135063000,
    1171063100,
    1154568000,
    1165065100,
    1168051000,
    1154551000,
    1114067000,
    1150061500,
    1111058000,
    1147055000,
    1154568000,
    1153054000,
    1154561000,
    1132051400,
    1147064000,
    1135069500,
    1168067500,
    1159060500,
    1165053000,
    1154567000,
    1171064700,
    1171052000,
    1111061500,
    1138069000,
    1141055500,
    1135060000,
    1165056000]

In [5]:
len(hdong_code_error)

134

In [11]:
# 결측치를 hdong_code_error 리스트의 원소들로 채워넣기
missing_indices = pharmacy[pharmacy['h_code'].isnull()].index
for index, value in zip(missing_indices, hdong_code_error):
    pharmacy.at[index, 'h_code'] = value

In [12]:
pharmacy.isnull().sum()

id                   0
name                 0
address              0
city_name            0
county_name          0
town_name            0
x                  134
y                  134
data_type            0
cleaned_address      0
h_code               0
dtype: int64

In [13]:
pharmacy.iloc[4]

id                                         7
name                                    국민약국
address            서울특별시 강서구 곰달래로 225, 1,2층 
city_name                                 서울
county_name                              강서구
town_name                                화곡동
x                                        NaN
y                                        NaN
data_type                                 약국
cleaned_address           서울특별시 강서구 곰달래로 225
h_code                          1150057000.0
Name: 4, dtype: object

#### x 좌표 결측치 처리

In [14]:
x_error = [
    126.8580318,
    126.8349216,
    126.9500687,
    127.0098500,
    126.9193479,
    126.9489800,
    127.0756013,
    127.1072604,
    126.9928779,
    126.8819679,
    126.9629880,
    126.9816553,
    126.9078149,
    126.9718828,
    126.8466719,
    127.0657785,
    126.9529188,
    127.0078453,
    127.0170403,
    127.0015857,
    127.0063224,
    127.0855946,
    127.0379729,
    127.0716058,
    127.0524597,
    126.9003320,
    127.0512686,
    127.0247865,
    126.8349216,
    127.0927002,
    127.1346966,
    127.0016822,
    127.0629614,
    126.9190816,
    127.1120415,
    126.8645649,
    126.9244940,
    126.9861682,
    127.0643054,
    126.8776792,
    127.0408461,
    127.0457672,
    126.9527595,
    127.0405332,
    126.9320642,
    127.1361648,
    127.0639372,
    126.9078149,
    127.0927002,
    127.0194894,
    126.9849030,
    126.9371605,
    126.9668857,
    127.0470790,
    127.0451556,
    126.9659909,
    127.1730577,
    126.9955133,
    126.8372423,
    126.8261369,
    126.8348939,
    127.0288249,
    126.8185936,
    127.0235331,
    127.0003589,
    127.0799007,
    127.1072604,
    127.1232611,
    127.1072604,
    127.0894904,
    126.8898406,
    127.1072604,
    127.1486654,
    126.8476921,
    127.0363876,
    126.9450362,
    127.0309552,
    127.0612970,
    127.0834812,
    127.0804744,
    127.0408461,
    127.0782455,
    127.1072604,
    127.1072604,
    127.1072604,
    126.9078399,
    127.0438072,
    126.9891333,
    127.0090714,
    126.9844827,
    127.1072604,
    126.9059864,
    127.1018445,
    126.9781310,
    126.9781310,
    127.0780909,
    127.0985302,
    127.0219296,
    127.0842018,
    127.0630609,
    126.9489800,
    126.9529188,
    127.0257084,
    127.0458375,
    126.9146175,
    126.9679867,
    126.9502937,
    127.0549036,
    127.1072604,
    126.9146175,
    127.0252675,
    127.0241229,
    126.8815811,
    127.0219296,
    126.8426550,
    126.9656863,
    126.8785570,
    126.9146175,
    126.8841835,
    126.8873951,
    127.0513599,
    126.8543984 ,
    127.0609192,
    127.0756013,
    126.9613540,
    127.0099968,
    126.9004586,
    127.1429237,
    127.1139664,
    126.9798606,
    126.9180011,
    126.9558605,
    127.0785138,
    127.0139480 ]

In [15]:
len(x_error)

134

In [16]:
missing_indices = pharmacy[pharmacy['x'].isnull()].index
for index, value in zip(missing_indices, x_error):
    pharmacy.at[index, 'x'] = value

In [17]:
pharmacy.isnull().sum()

id                   0
name                 0
address              0
city_name            0
county_name          0
town_name            0
x                    0
y                  134
data_type            0
cleaned_address      0
h_code               0
dtype: int64

#### y 좌표 결측치 처리

In [18]:
y_error = [
    37.5318488,
    37.5578848,
    37.5338712,
    37.5419333,
    37.6357288,
    37.5413405,
    37.4926645,
    37.4976183,
    37.5614912,
    37.4770424,
    37.5475870,
    37.4846358,
    37.5747271,
    37.5201533,
    37.5324773,
    37.6239862,
    37.5529598,
    37.5044921,
    37.5906601,
    37.5796275,
    37.5623086,
    37.5097446,
    37.4843263,
    37.4851086,
    37.5931680,
    37.5866386,
    37.5127264,
    37.5665508,
    37.5578848,
    37.6163597,
    37.5248028,
    37.5819294,
    37.4938697,
    37.6386516,
    37.5312314,
    37.5082533,
    37.6354766,
    37.5616541,
    37.5170204,
    37.5497940,
    37.5167495,
    37.6519602,
    37.5445786,
    37.6018543,
    37.4790260,
    37.5366672,
    37.6362190,
    37.5747271,
    37.6163597,
    37.6072048,
    37.5639308,
    37.5591598,
    37.5290927,
    37.6680424,
    37.4934007,
    37.5691438,
    37.5664715,
    37.5619014,
    37.5596114,
    37.5592737,
    37.5584039,
    37.5566034,
    37.5671210,
    37.5032162,
    37.5707113,
    37.6670475,
    37.4976183,
    37.4805094,
    37.4976183,
    37.5803080,
    37.5056084,
    37.4976183,
    37.5314130,
    37.5656521,
    37.5012700,
    37.5109209,
    37.6050947,
    37.6500621,
    37.4857650,
    37.5912181,
    37.5167495,
    37.6118425,
    37.4976183,
    37.4976183,
    37.4976183,
    37.5581853,
    37.5801098,
    37.5646140,
    37.5900036,
    37.5643996,
    37.4976183,
    37.5157568,
    37.4880245,
    37.4884496,
    37.4884496,
    37.6115688,
    37.5425315,
    37.5658796,
    37.5120894,
    37.5772275,
    37.5413405,
    37.5529598,
    37.4920742,
    37.5749667,
    37.4503622,
    37.5675159,
    37.5592794,
    37.6803112,
    37.4976183,
    37.4503622,
    37.4704214,
    37.5284222,
    37.4819131,
    37.5658796,
    37.5433574,
    37.5674148,
    37.5363941,
    37.4503622,
    37.4930720,
    37.4577246,
    37.6527733,
    37.5139862,
    37.6518073,
    37.4926645,
    37.5080534,
    37.4840976,
    37.4538129,
    37.4812025,
    37.5273835 ,
    37.5708345,
    37.6350853,
    37.5586219,
    37.6259743,
    37.5066310]

In [19]:
len(y_error)

134

In [20]:
missing_indices = pharmacy[pharmacy['y'].isnull()].index
for index, value in zip(missing_indices, y_error):
    pharmacy.at[index, 'y'] = value

In [21]:
pharmacy.isnull().sum()

id                 0
name               0
address            0
city_name          0
county_name        0
town_name          0
x                  0
y                  0
data_type          0
cleaned_address    0
h_code             0
dtype: int64

In [22]:
pharmacy.iloc[4]

id                                         7
name                                    국민약국
address            서울특별시 강서구 곰달래로 225, 1,2층 
city_name                                 서울
county_name                              강서구
town_name                                화곡동
x                                 126.858032
y                                  37.531849
data_type                                 약국
cleaned_address           서울특별시 강서구 곰달래로 225
h_code                          1150057000.0
Name: 4, dtype: object

In [26]:
pharmacy = pharmacy.drop(["id", "cleaned_address"], axis=1)

In [27]:
pharmacy

,name,address,city_name,county_name,town_name,x,y,data_type,h_code
0,청담장수 한약국,"서울특별시 강남구 도산대로89길 3,",서울,강남구,청담동,37.525057,37.525057,약국,1.168056e+09
1,대도약국,"서울특별시 종로구 우정국로 38-1, 1층",서울,종로구,견지동,37.572483,37.572483,약국,1.111062e+09
2,메디팜정연약국,"서울특별시 강남구 강남대로 256, 103호",서울,강남구,도곡동,37.486487,37.486487,약국,1.168066e+09
3,진선약국,"서울특별시 강남구 선릉로 424,",서울,강남구,대치동,37.502910,37.502910,약국,1.168063e+09
4,국민약국,"서울특별시 강서구 곰달래로 225, 1,2층",서울,강서구,화곡동,126.858032,37.531849,약국,1.150057e+09
...,...,...,...,...,...,...,...,...,...
5550,고바우약국,"서울특별시 송파구 마천로41길 21,",서울,송파구,마천동,37.497121,37.497121,약국,1.171055e+09
5551,선약국,"서울특별시 강서구 강서로 194-9, 1층",서울,강서구,화곡동,37.544048,37.544048,약국,1.150062e+09
5552,최고약국,"서울특별시 서초구 서초대로 351, 최상산부인과 202호",서울,서초구,서초동,37.495795,37.495795,약국,1.165053e+09
5553,온누리큰사랑약국,"서울특별시 강서구 강서로 64,",서울,강서구,화곡동,37.533504,37.533504,약국,1.150059e+09


### csv 파일로 저장

In [28]:
pharmacy.to_csv("./pharmacies_final.csv", index=False)